## Neural Network

In [9]:
import sys
sys.path.append('../../../../')

# Import libraries
from utils.load import load
from utils.score import fold_cross_validate
from utils.print import print_cross_validation_scores
from constant.columns import FEATURES, LABEL

import pandas as pd
import keras as keras
from keras import activations, backend as K
from keras.wrappers import scikit_learn
from keras.models import Sequential
from keras.layers import Dense
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split

In [10]:
# Read dataset
df = pd.read_csv(f'./source/dataset_smote.csv')
X, y = df[FEATURES], df[LABEL]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

#### Cross Validation

In [11]:
def f1_score(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

# Define a sklearn-compatible classifier based on our Keras model
class KerasBinaryClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, model):
        self.model = model
    def fit(self, X, y):
        self.model.compile(loss='binary_crossentropy', optimizer='adam')
        self.model.fit(X, y, epochs=150, batch_size=10, verbose=0)
        return self
    def predict(self, X):
        Y_pred = self.model.predict(X)
        Y_pred = (Y_pred >= 0.5).astype(int)
        return Y_pred.flatten()

def create_model():
    model = Sequential()
    model.add(Dense(len(FEATURES), input_dim=len(FEATURES), activation=activations.relu))
    model.add(Dense(1, activation=activations.sigmoid))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [12]:
# Define the model
clf_cv = scikit_learn.KerasClassifier(build_fn=create_model, epochs=50, batch_size=32, verbose=0)
scores = fold_cross_validate(clf_cv, X_test, y_test)

# Print scores
print_cross_validation_scores(scores)

C:\Users\dell\AppData\Local\Temp\ipykernel_29472\1000466164.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  clf_cv = scikit_learn.KerasClassifier(build_fn=create_model, epochs=50, batch_size=32, verbose=0)


MACRO:
Precision: 0.7754 (0.0485)
Recall: 0.7658 (0.0510)
F1 score: 0.7619 (0.0527)
---------------------------------
MICRO:
Precision: 0.7652 (0.0511)
Recall: 0.7652 (0.0511)
F1 score: 0.7652 (0.0511)
---------------------------------
